- 기본 비교: YAKE/RAKE/TextRank/KeyBERT
- 핵심 지표: F1@10, NDCG@10, Diversity, p95 latency(원하면 groupby 후 quantile), Jaccard variance(옵션)

In [ ]:
# pip install yake rake-nltk keybert sentence-transformers networkx langdetect scikit-learn pandas numpy python-dotenv
from __future__ import annotations
import os, re, json, time, random, math
from pathlib import Path
from typing import List, Dict, Any, Tuple, Callable, Optional
from dotenv import load_dotenv
import numpy as np
import pandas as pd

from langdetect import detect
import networkx as nx

import yake
from rake_nltk import Rake
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

load_dotenv()
set_seed(42)

print(Path.cwd())


/home/user/workspace/redfin/redfin_label_api/notebooks


In [ ]:
# =========================
# 실험 공통 설정 (CONFIG)
# =========================

# ---------- 재현성 ----------
SEED: int = 42                  # 랜덤 시드 고정 (YAKE/RAKE에는 영향 적지만 KeyBERT 등에 일관성 부여)
PYTHONHASHSEED: str = str(SEED) # 파이썬 해시 시드 고정 (문자열로 환경변수에 반영)
os.environ["PYTHONHASHSEED"] = PYTHONHASHSEED

# ---------- 데이터 경로 ----------
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
INPUT_JSONL: str  = DATA_DIR / "ai_news.jsonl"             # 라인-델리미티드 JSONL 입력(각 행: {id,title,description,content,...})
GOLD_JSON: str    = DATA_DIR / "gold_keywords.json"        # 골드 키워드 맵 {doc_id: [kw1, kw2, ...]} (없으면 평가 생략 가능)
ROWS_CSV: str     = DATA_DIR / "keyword_eval_rows.csv"     # 문서별·알고리즘별 원시 결과 저장 경로
REPORT_CSV: str   = DATA_DIR / "keyword_eval_report.csv"   # 알고리즘별 평균 리포트 저장 경로

# ---------- 평가 공통 ----------
TOP_K: int        = 10          # 각 알고리즘이 반환할 키워드 상위 K개(Precision/Recall/NDCG 대상)
EVAL_K: int       = 10          # 평가 시 사용할 K값(보통 TOP_K와 동일하게 둠)
CONSISTENCY_RUNS: int = 2       # 동일 입력 반복 실행 횟수(일관성/변동성 측정; 0이면 비활성)

# ---------- 대상 알고리즘 선택 ----------
# 사용 가능한 값: "yake", "rake", "textrank", "keybert"
METHODS: list[str] = ["yake", "rake", "textrank", "keybert"]

# ---------- KeyBERT/임베딩 설정 ----------
EMBED_MODEL_NAME: str = "sentence-transformers/all-MiniLM-L6-v2"  # 경량·속도 균형(의미 비교용)
KEYBERT_NR_CANDIDATES: int = 20    # 후보군 개수 (너무 크면 느려짐)
KEYBERT_DIVERSITY: float = 0.6     # MaxSum/Maximal marginal relevance 유사 개념(높을수록 다양성↑)

# ---------- YAKE 하이퍼파라미터 ----------
YAKE_MAX_NGRAM: int = 3           # 1~3gram까지 고려
YAKE_DEDUP_LIM: float = 0.9       # 유사 키워드 중복 제거 임계값(낮을수록 공격적 중복 제거)
YAKE_WINDOW: int = 2              # 윈도 크기(공동 발생 고려)

# ---------- RAKE 하이퍼파라미터 ----------
RAKE_MAX_WORDS: int = 3           # n-gram 최대 단어 수(영문 RSS 기사에는 1~3 추천)

# ---------- TextRank 하이퍼파라미터 ----------
TR_WINDOW: int = 2                 # 공출현 윈도 크기(너무 크면 그래프 과밀/속도저하)

# ---------- 로깅/성능 ----------
LOG_P95_LATENCY: bool = True       # 그룹 요약 시 p95 지연 계산 포함 여부
SAVE_INTERMEDIATE: bool = True     # 중간 산출물(행 단위 결과 CSV) 저장 여부

# ---------- 운영상 권고 ----------
#  - KeyBERT는 첫 로드 시 임베딩 모델 다운로드로 시간이 다소 소요될 수 있음.
#  - CPU-only 환경에서도 동작하나, 데이터가 많으면 keybert 단계가 병목이 될 수 있음.
#  - TOP_K/EVAL_K는 동일하게 두는 것을 권장(리포트 해석 단순화).
#  - CONSISTENCY_RUNS는 2~3 수준만으로도 충분히 변동성 추세 파악 가능.

def print_config():
    """현재 실험 설정값을 표 형태로 간단 요약 출력"""
    from pprint import pprint
    cfg = {
        "SEED": SEED,
        "INPUT_JSONL": INPUT_JSONL,
        "GOLD_JSON": GOLD_JSON,
        "TOP_K / EVAL_K": (TOP_K, EVAL_K),
        "CONSISTENCY_RUNS": CONSISTENCY_RUNS,
        "METHODS": METHODS,
        "EMBED_MODEL_NAME": EMBED_MODEL_NAME,
        "KEYBERT_NR_CANDIDATES": KEYBERT_NR_CANDIDATES,
        "KEYBERT_DIVERSITY": KEYBERT_DIVERSITY,
        "YAKE": {"MAX_NGRAM": YAKE_MAX_NGRAM, "DEDUP_LIM": YAKE_DEDUP_LIM, "WINDOW": YAKE_WINDOW},
        "RAKE": {"MAX_WORDS": RAKE_MAX_WORDS},
        "TEXTRANK": {"WINDOW": TR_WINDOW},
        "LOG_P95_LATENCY": LOG_P95_LATENCY,
        "SAVE_INTERMEDIATE": SAVE_INTERMEDIATE,
        "OUTPUT_ROWS_CSV": ROWS_CSV,
        "OUTPUT_REPORT_CSV": REPORT_CSV,
    }
    pprint(cfg)

print_config()

{'CONSISTENCY_RUNS': 2,
 'EMBED_MODEL_NAME': 'sentence-transformers/all-MiniLM-L6-v2',
 'GOLD_JSON': 'gold_keywords.json',
 'INPUT_JSONL': 'ai_news.jsonl',
 'KEYBERT_DIVERSITY': 0.6,
 'KEYBERT_NR_CANDIDATES': 20,
 'LOG_P95_LATENCY': True,
 'METHODS': ['yake', 'rake', 'textrank', 'keybert'],
 'OUTPUT_REPORT_CSV': 'keyword_eval_report.csv',
 'OUTPUT_ROWS_CSV': 'keyword_eval_rows.csv',
 'RAKE': {'MAX_WORDS': 3},
 'SAVE_INTERMEDIATE': True,
 'SEED': 42,
 'TEXTRANK': {'WINDOW': 2},
 'TOP_K / EVAL_K': (10, 10),
 'YAKE': {'DEDUP_LIM': 0.9, 'MAX_NGRAM': 3, 'WINDOW': 2}}


In [10]:
# 데이터 I/O 유틸 (JSONL)
def read_jsonl(path: str) -> List[Dict[str, Any]]:
    out = []
    with open(path, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if not ln: 
                continue
            try:
                out.append(json.loads(ln))
            except json.JSONDecodeError:
                pass
    return out

def write_jsonl(path: str, rows: List[Dict[str, Any]]):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")


In [11]:
# 통일 전처리
# pip install langdetect kiwipiepy spacy

def detect_language(input_text):
    from langdetect import detect, detect_langs
    
    try:
        langs = detect(input_text) # ko
        return langs
    except Exception as e:
        print(f"[Error] 언어 감지 실패: {e}")
        return None

In [12]:
# 전처리 (영문 전제) + 토크나이즈
_WORD_RE = re.compile(r"[A-Za-z][A-Za-z\-\.]{1,29}")

def normalize_text(title: str, description: str = "", content: str = "") -> str:
    t = f"{(title or '').strip()} {(description or '').strip()} {(content or '').strip()}".strip()
    t = re.sub(r"\s+", " ", t)
    return t

def tokenize_en_simple(text: str) -> List[str]:
    return [m.group(0).lower() for m in _WORD_RE.finditer(text)]

def ngrams(tokens: List[str], n_min=1, n_max=3) -> List[str]:
    out = []
    for n in range(n_min, n_max + 1):
        for i in range(len(tokens) - n + 1):
            phrase = " ".join(tokens[i:i+n])
            if not re.search(r"https?://|\d{5,}", phrase):
                out.append(phrase)
    return out


In [13]:
# 키워드 추출기 팩토리 (YAKE/RAKE/TextRank/KeyBERT)

# 1. YAKE
def make_yake(top_k: int = 10, max_ngram: int = 3, dedup_lim: float = 0.9, window: int = 2):
    extractor = yake.KeywordExtractor(
        lang="en",
        n=max_ngram,
        top=top_k,
        dedupLim=dedup_lim,
        windowsSize=window
    )
    def _extract(text: str) -> List[str]:
        return [kw for kw, _ in extractor.extract_keywords(text)]
    return _extract

# 2. RAKE
def make_rake(top_k: int = 10, max_words: int = 3):
    rake = Rake(min_length=1, max_length=max_words)  # 영어 불용어 내장
    def _extract(text: str) -> List[str]:
        rake.extract_keywords_from_text(text)
        phrases = [p for p, _ in rake.get_ranked_phrases_with_scores()]
        return phrases[:top_k]
    return _extract

# 3. TextRank
def make_textrank(top_k: int = 10, window: int = 2):
    """
    경량 TextRank: 공출현 그래프 → PageRank
    """
    def _extract(text: str) -> List[str]:
        toks = tokenize_en_simple(text)
        if not toks:
            return []
        words = ngrams(toks, 1, 1)
        if not words:
            return []
        G = nx.Graph()
        G.add_nodes_from(set(words))
        for i in range(len(words) - window):
            for j in range(1, window + 1):
                a, b = words[i], words[i+j]
                if a != b:
                    if G.has_edge(a, b):
                        G[a][b]["weight"] += 1.0
                    else:
                        G.add_edge(a, b, weight=1.0)
        pr = nx.pagerank(G, alpha=0.85, weight="weight")
        ranked = sorted(pr.items(), key=lambda x: x[1], reverse=True)
        return [w for w, _ in ranked[:top_k]]
    return _extract

# 4. KeyBERT
_EMBED = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
def make_keybert(top_k: int = 10, diversity: float = 0.6, nr_candidates: int = 20):
    kb = KeyBERT(model=_EMBED)
    def _extract(text: str) -> List[str]:
        pairs = kb.extract_keywords(
            text,
            keyphrase_ngram_range=(1,3),
            top_n=top_k,
            use_maxsum=True,
            nr_candidates=nr_candidates,
            diversity=diversity,
        )
        return [k for k, _ in pairs]
    return _extract

EXTRACTOR_FACTORIES: Dict[str, Callable[..., Callable[[str], List[str]]]] = {
    "yake": make_yake,
    "rake": make_rake,
    "textrank": make_textrank,
    "keybert": make_keybert,
}

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# 평가 지표 함수
def precision_recall_f1_at_k(pred: List[str], gold: List[str], k: int = 10) -> Tuple[float, float, float]:
    p = set(pred[:k]); g = set(gold)
    tp = len(p & g)
    P = tp / max(1, len(p))
    R = tp / max(1, len(g))
    F1 = 2*P*R/(P+R) if (P+R) > 0 else 0.0
    return P, R, F1

def ndcg_at_k(pred: List[str], gold: List[str], k: int = 10) -> float:
    gset = set(gold); dcg = 0.0
    for i, w in enumerate(pred[:k], start=1):
        rel = 1.0 if w in gset else 0.0
        dcg += rel / math.log2(i + 1)
    ideal = sum(1.0 / math.log2(i + 1) for i in range(1, min(k, len(gold)) + 1))
    return dcg / max(ideal, 1e-9)

def diversity_cosine(phrases: List[str]) -> float:
    if len(phrases) < 2:
        return 1.0
    embs = _EMBED.encode(phrases, normalize_embeddings=True)
    sim = cosine_similarity(embs)
    upper = sim[np.triu_indices(len(phrases), 1)]
    return 1.0 - float(np.mean(upper))  # 높을수록 다양

In [15]:
# 단일 문서 실행/평가 + 반복 변동성(Jaccard 분산)
def jaccard(a: List[str], b: List[str], k: int = 10) -> float:
    A, B = set(a[:k]), set(b[:k])
    u = len(A | B)
    return len(A & B)/u if u else 1.0

def run_extract(extractor: Callable[[str], List[str]], text: str) -> Tuple[List[str], Dict[str, Any]]:
    t0 = time.time()
    keys = extractor(text) if text else []
    dt = (time.time() - t0) * 1000.0  # ms
    return keys, {"latency_ms": dt, "out_len": len(keys)}

def eval_one(pred: List[str], gold: List[str], k: int = 10) -> Dict[str, float]:
    P, R, F1 = precision_recall_f1_at_k(pred, gold, k=k)
    ndcg = ndcg_at_k(pred, gold, k=k)
    div  = diversity_cosine(pred[:k])
    return {"P@k": P, "R@k": R, "F1@k": F1, "NDCG@k": ndcg, "Diversity": div}

def consistency_variance(extractor: Callable[[str], List[str]], text: str, runs: int = 3, k: int = 10) -> float:
    outs = []
    for _ in range(runs):
        keys, _ = run_extract(extractor, text)
        outs.append(keys)
    if len(outs) < 2:
        return 0.0
    js = []
    for i in range(len(outs)-1):
        js.append(jaccard(outs[i], outs[i+1], k=k))
    return float(np.var(js))


In [16]:
# 배치 실행 러너 (여러 알고리즘 × 데이터셋)
def run_batch(
    data: List[Dict[str, Any]],
    gold_map: Dict[str, List[str]] | None,
    methods: List[str],
    top_k: int = 10,
    repeat_consistency_runs: int = 0,
    seed: int = 42,
) -> pd.DataFrame:
    """
    data: 각 항목에 고유 id가 존재하면 gold_map과 연결하기 좋음. (없으면 index 사용)
    gold_map: {doc_id: [gold keywords]} 또는 None(평가 생략)
    methods: ["yake","rake","textrank","keybert"] 중 선택
    """
    set_seed(seed)
    rows = []
    # 미리 추출기 빌드
    exts: Dict[str, Callable[[str], List[str]]] = {}
    for m in methods:
        if m == "yake":
            exts[m] = make_yake(top_k=top_k)
        elif m == "rake":
            exts[m] = make_rake(top_k=top_k)
        elif m == "textrank":
            exts[m] = make_textrank(top_k=top_k)
        elif m == "keybert":
            exts[m] = make_keybert(top_k=top_k)

    for idx, item in enumerate(data):
        did = item.get("id", idx)
        text = normalize_text(item.get("title",""), item.get("description",""), item.get("content",""))
        for m in methods:
            keys, info = run_extract(exts[m], text)
            rec = {
                "doc_id": did,
                "method": m,
                "latency_ms": info["latency_ms"],
                "pred_kws": keys[:top_k],
                "pred_len": info["out_len"],
            }
            # 평가
            if gold_map and did in gold_map:
                gold = gold_map[did]
                met = eval_one(keys, gold, k=top_k)
                rec.update(met)
            # 일관성(선택)
            if repeat_consistency_runs > 0:
                var = consistency_variance(exts[m], text, runs=repeat_consistency_runs, k=top_k)
                rec["jaccard_var"] = var
            rows.append(rec)
    return pd.DataFrame(rows)


In [ ]:
DATA_PATH = "articles.jsonl"
data = read_jsonl(DATA_PATH)

# 골드 키워드 매핑 예시 (실험 전 사람이 만든 레이블) : {doc_id: [kw1, kw2, ...]}
# 실제 사용 시 별도 파일에서 로딩하세요.
gold_map = {
    # "123": ["openai","gpt","regulation","nvidia","microsoft"],
}

methods = ["yake","rake","textrank","keybert"]
df = run_batch(
    data=data,
    gold_map=gold_map,              # 골드 없으면 None
    methods=methods,
    top_k=10,
    repeat_consistency_runs=2,      # 일관성 측정(선택). 0이면 비활성
    seed=42,
)

# 결과 확인
df.head()

In [ ]:
# 요약 리포트
agg_cols = ["latency_ms","P@k","R@k","F1@k","NDCG@k","Diversity","jaccard_var"]
report = (
    df
    .groupby("method")[agg_cols]
    .mean()
    .sort_values("F1@k", ascending=False)
    .round(4)
)
report

In [ ]:
# 결과 저장
